<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/2s2024/TIC/Aula_11_Series_temporais_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="left">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

# **Séries Temporais I**

Uma série temporal é uma sequência de observações ou medidas feitas em um mesmo objeto ou fenômeno em intervalos de tempo igualmente espaçados. Essas observações são coletadas e registradas em função do tempo. Em outras palavras, uma série temporal é uma coleção de dados organizados ao longo do tempo, o que permite analisar tendências, padrões, sazonalidades e outros comportamentos que possam ocorrer em diferentes escalas de tempo.

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_series_temporais_1.png' width="600">

As séries temporais são amplamente usadas em várias disciplinas, como econometria, meteorologia, finanças, engenharia e muitas outras, para analisar e prever o comportamento de fenômenos que evoluem com o tempo. Elas desempenham um papel fundamental na modelagem estatística e na tomada de decisões com base no histórico de dados.

# Estacionariedade

A média e a variância de processos estacionários são constantes ao longo do tempo, intuitivamente isto significa que as observações se desenvolvem aleatoriamente entorno de um certo nível com variabilidade constante.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Séries temporais
df_1 = pd.read_excel("https://www.orlandoalbarracin.com.br/phyton/Series.xlsx", sheet_name="dolar") #Economial UOL - diária
df_2 = pd.read_excel("https://www.orlandoalbarracin.com.br/phyton/Series.xlsx", sheet_name="Exportações") #IBGE - Trimestral


In [ ]:
#@markdown Gráfico das séries
import matplotlib.dates as mdates

index_dates1 = pd.date_range(start='7/2/2018', end='12/31/2019', freq='B')
index_dates2 = pd.date_range(start='3/1/1996', end='12/31/2021', freq='3M')

df_1.index=index_dates1
df_2.index=index_dates2

fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(12,4))

ax1.plot(df_1.index.to_pydatetime(), df_1["Var"], c='orange')
ax1.xaxis.set_major_locator(mdates.MonthLocator([4,8,12]))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b-%y'))
ax1.set_title('Variação diária da cotação do dólar',fontdict = {'fontsize' : 12})
ax1.set_xlabel('\n Série "estacionária"',fontdict = {'fontsize' : 11},color="olive")

ax1.set_ylim([-4, 4]);

ax2.plot(df_2.index, df_2.Exportações)
ax2.set_title('Exportação de bens e serviços \n Valores a preços correntes (Milhões de Reais)',fontdict = {'fontsize' : 12})
ax2.set_xlabel('\n Série não estacionária \n (com tedência)',fontdict = {'fontsize' : 11},color="darkblue")


### Teste Augmented Dickey-Fuller (ADF)

O teste de Dickey-Fuller Aumentado (ADF) é um teste estatístico comumente usado para verificar a presença de uma tendência estocástica em uma série temporal.

$$H_0: \text{A série não é estacionária}$$
$$H_1: \text{A série é estacionária}$$

Se o valor-p for menor que um determinado nível de significância (geralmente 0,05), então rejeitamos a hipótese nula e concluímos que a série é estacionária, ou seja, não possui uma tendência estocástica significativa.

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_1.Var)
print('p-value: %f' % result[1])

In [ ]:
#  Exportações
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_2.Exportações)
print('p-value: %f' % result[1])

#Decomposição de séries temporais

A decomposição de uma série temporal é o processo de separar a série em diferentes componentes: tendência, sazonalidade e componente aleatório, isto permite a detecção de padrões, identificação de tendências e sazonalidades, e auxiliando na previsão de valores da série.

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_series_temporais_2.png' width="600">

*   **Tendência**  direção geral dos dados ao longo do tempo, indica se a série está aumentando, diminuindo ou permanecendo estável ao longo de um período de tempo.

*   **Sazonalidade** refere-se a padrões que se repetem em períodos fixos de tempo, como estações do ano, dias da semana ou meses.

*   **Componente aleatório (erro)**: representa a variabilidade não explicada pela tendência e sazonalidade.



# Australian beer production e Airline Passenger Numbers

Produção mensal de cerveja na Austrália: megalitros. Não inclui bebidas com teor alcoólico inferior a 1,15.

###1.  Verifique visualmente se as séries apresentam tendências e/ou sazonalidades

In [ ]:
# Lendo os dados
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/ejgao/Time-Series-Datasets/master/monthly-beer-production-in-austr.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv")
df = df.iloc[48:252,:].reset_index(drop=True)

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(df["Monthly beer production"])
plt.title("Monthly beer production")

plt.subplot(1,2,2)
plt.plot(df2["Passengers"])
plt.title("Airline Passenger Numbers")

plt.show()

Observação:

*   **Monthly beer production:**
A variação sazonal parece constante; ela não muda quando o valor da série temporal aumenta. Devemos usar o modelo aditivo.
$Valor = Tendência +  Sazonalidade + Erro$

*   **Airline Passenger Numbers**
À medida que a série temporal aumenta em magnitude, a variação sazonal também aumenta. Nesse caso, devemos usar o modelo multiplicativo. $ Valor = Tendência \times Sazonalidade \times Erro$



### 2. Faça uma análise descritiva da produção de cerveja por mês. Em qual mês a produção é maior e em qual é menor?

In [ ]:
# Vamos ver o formato da base de dados!
df

In [ ]:
df[['Ano', 'Mês']] = df['Month'].apply(lambda x: pd.Series(x.split('-')))
df.drop("Month", axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# Produção por mês!
sns.boxplot(x="Mês", y="Monthly beer production", data=df)
plt.show()

###3. Ajuste um modelo de regressão linear para a série produção de cerveja. Faça as previsões para o próximo ano. Comente!

In [ ]:
# Vamos criar uma coluna com o tempo para ajustar o modelo de regressão

df["time"]=np.arange(len(df))
df.head()

In [ ]:
import statsmodels.formula.api as sm

model = sm.ols(formula='df["Monthly beer production"] ~ time', data=df)
result = model.fit()
print(result.summary())

In [ ]:
# previsões 12 meses

new_times = pd.DataFrame()
new_times['time'] = np.arange(len(df), len(df) + 12)

In [ ]:
new_times['prediction'] = result.predict(new_times)
new_times.head()

In [ ]:
plt.plot(df["Monthly beer production"])
sns.lineplot(x='time', y='prediction', data=new_times, color='red')

### 4. Faça a decomposição da série. Comente!

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition=seasonal_decompose(df["Monthly beer production"],
                                 model='additive',
                                 period=12,
                                 extrapolate_trend='freq')
decomposition.plot();

### 5. Analise os decompostos (tend+saz+resid) dos valores da série em janeiro e dezembro de 1961.

In [ ]:
# Obter os componentes da decomposição

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

In [ ]:
df["trend"]=trend
df["seasonal"]=seasonal
df["residual"]=residual

In [ ]:
df.head()

In [ ]:
df[(df.Ano=="1961") & (df.Mês=="01")]

In [ ]:
df[(df.Ano=="1961") & (df.Mês=="12")]

###6. Verifique estatísticamente se os resíduos obtidos da decomposição da série são estacionários. Use $\alpha=0,05$.

In [ ]:
plt.plot(df.residual);

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df.residual)
print('p-value: %f' % result[1])

## Exercício

Faça as análises anteriores para prever o número de bicicletas alugadas ocasionalmente (coluna `casual`) usando o conjunto de dados [Bike Sharing](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset), disponível no link:

https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Datasets/data_bike_sharing_day.csv

Verifique se há tendência e/ou sazonalidade, crie um modelo de regressão linear para prever o número de bicicletas alugadas casualmente, e faça a decomposição da série temporal.